# Test Weight Schedules for Regulation Changes

**Goal:** Find optimal weight schedule for combining:
- Baseline (2021 standings)
- Testing (2022 pre-season directionality)
- Current season (2022 actual results)

**Test schedules:**
- Conservative → Ultra Aggressive → Extreme → Insane
- Control groups: testing_only, baseline_only

**Validation:** 2021→2022 regulation change (proxy for 2026)

In [2]:
import json
import warnings
from pathlib import Path

import fastf1
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from scipy.stats import spearmanr

warnings.filterwarnings('ignore')

import logging

logging.getLogger("fastf1").setLevel(logging.ERROR)

# Enable cache
cache_dir = Path("../data/raw/.fastf1_cache")
cache_dir.mkdir(parents=True, exist_ok=True)
fastf1.Cache.enable_cache(str(cache_dir))

print("✓ Setup complete")

✓ Setup complete


## Step 1: Define Weight Schedules

In [3]:
# Format: race_number -> (baseline_weight, testing_weight, current_season_weight)

SCHEDULES = {
    'conservative': {
        1: (0.70, 0.25, 0.05),
        2: (0.65, 0.25, 0.10),
        3: (0.60, 0.20, 0.20),
        4: (0.50, 0.20, 0.30),
        5: (0.40, 0.15, 0.45),
        6: (0.30, 0.10, 0.60),
        7: (0.20, 0.05, 0.75),
        8: (0.15, 0.00, 0.85),
        9: (0.10, 0.00, 0.90),
        10: (0.10, 0.00, 0.90)
    },
    'moderate': {
        1: (0.60, 0.30, 0.10),
        2: (0.50, 0.25, 0.25),
        3: (0.40, 0.20, 0.40),
        4: (0.30, 0.15, 0.55),
        5: (0.20, 0.10, 0.70),
        6: (0.15, 0.05, 0.80),
        7: (0.10, 0.00, 0.90),
        8: (0.10, 0.00, 0.90),
        9: (0.10, 0.00, 0.90),
        10: (0.10, 0.00, 0.90)
    },
    'aggressive': {
        1: (0.50, 0.30, 0.20),
        2: (0.40, 0.20, 0.40),
        3: (0.25, 0.15, 0.60),
        4: (0.15, 0.10, 0.75),
        5: (0.10, 0.05, 0.85),
        6: (0.10, 0.00, 0.90),
        7: (0.10, 0.00, 0.90),
        8: (0.10, 0.00, 0.90),
        9: (0.10, 0.00, 0.90),
        10: (0.10, 0.00, 0.90)
    },
    'very_aggressive': {
        1: (0.40, 0.30, 0.30),
        2: (0.30, 0.20, 0.50),
        3: (0.20, 0.10, 0.70),
        4: (0.10, 0.05, 0.85),
        5: (0.10, 0.00, 0.90),
        6: (0.10, 0.00, 0.90),
        7: (0.10, 0.00, 0.90),
        8: (0.10, 0.00, 0.90),
        9: (0.10, 0.00, 0.90),
        10: (0.10, 0.00, 0.90)
    },
    'ultra_aggressive': {
        1: (0.35, 0.25, 0.40),
        2: (0.20, 0.15, 0.65),
        3: (0.10, 0.05, 0.85),
        4: (0.05, 0.00, 0.95),
        5: (0.05, 0.00, 0.95),
        6: (0.05, 0.00, 0.95),
        7: (0.05, 0.00, 0.95),
        8: (0.05, 0.00, 0.95),
        9: (0.05, 0.00, 0.95),
        10: (0.05, 0.00, 0.95)
    },
    'extreme': {
        1: (0.30, 0.20, 0.50),
        2: (0.15, 0.10, 0.75),
        3: (0.05, 0.00, 0.95),
        4: (0.05, 0.00, 0.95),
        5: (0.05, 0.00, 0.95),
        6: (0.05, 0.00, 0.95),
        7: (0.05, 0.00, 0.95),
        8: (0.05, 0.00, 0.95),
        9: (0.05, 0.00, 0.95),
        10: (0.05, 0.00, 0.95)
    },
    'insane': {
        1: (0.25, 0.15, 0.60),
        2: (0.10, 0.05, 0.85),
        3: (0.05, 0.00, 0.95),
        4: (0.00, 0.00, 1.00),
        5: (0.00, 0.00, 1.00),
        6: (0.00, 0.00, 1.00),
        7: (0.00, 0.00, 1.00),
        8: (0.00, 0.00, 1.00),
        9: (0.00, 0.00, 1.00),
        10: (0.00, 0.00, 1.00)
    },
    'testing_only': {
        1: (0.70, 0.30, 0.00),
        2: (0.70, 0.30, 0.00),
        3: (0.70, 0.30, 0.00),
        4: (0.70, 0.30, 0.00),
        5: (0.70, 0.30, 0.00),
        6: (0.70, 0.30, 0.00),
        7: (0.70, 0.30, 0.00),
        8: (0.70, 0.30, 0.00),
        9: (0.70, 0.30, 0.00),
        10: (0.70, 0.30, 0.00)
    },
    'baseline_only': {
        1: (1.00, 0.00, 0.00),
        2: (1.00, 0.00, 0.00),
        3: (1.00, 0.00, 0.00),
        4: (1.00, 0.00, 0.00),
        5: (1.00, 0.00, 0.00),
        6: (1.00, 0.00, 0.00),
        7: (1.00, 0.00, 0.00),
        8: (1.00, 0.00, 0.00),
        9: (1.00, 0.00, 0.00),
        10: (1.00, 0.00, 0.00)
    }
}

print(f"✓ Defined {len(SCHEDULES)} schedules")
print("\nKey differences:")
print("  conservative:     Race 1 = 5% current")
print("  very_aggressive:  Race 1 = 30% current")
print("  ultra_aggressive: Race 1 = 40% current")
print("  extreme:          Race 1 = 50% current")
print("  insane:           Race 1 = 60% current, Race 4+ = 100% current")

# Visualize
fig = go.Figure()

for schedule_name, weights in SCHEDULES.items():
    races = sorted(weights.keys())
    current_weights = [weights[r][2] for r in races]
    
    fig.add_trace(go.Scatter(
        x=races,
        y=current_weights,
        mode='lines+markers',
        name=schedule_name,
        line=dict(width=2)
    ))

fig.update_layout(
    title='Weight Schedules: Current Season Weight by Race',
    xaxis_title='Race Number',
    yaxis_title='Current Season Weight',
    height=500,
    hovermode='x unified'
)
fig.show()

✓ Defined 9 schedules

Key differences:
  conservative:     Race 1 = 5% current
  very_aggressive:  Race 1 = 30% current
  ultra_aggressive: Race 1 = 40% current
  extreme:          Race 1 = 50% current
  insane:           Race 1 = 60% current, Race 4+ = 100% current


## Step 2: Helper Functions

In [4]:
def extract_testing_directionality(year):
    """Extract testing directionality."""
    print(f"Extracting {year} testing...")
    
    try:
        schedule = fastf1.get_event_schedule(year)
        testing_events = schedule[schedule['EventFormat'] == 'testing']
        
        if len(testing_events) == 0:
            return None
        
        team_data = {}
        
        for idx, event in testing_events.iterrows():
            for day in [1, 2, 3]:
                try:
                    session = fastf1.get_session(year, event['EventName'], day)
                    session.load(laps=True, telemetry=True)
                    
                    laps = session.laps[
                        (session.laps['IsAccurate'] == True) & 
                        (session.laps['TrackStatus'] == '1')
                    ]
                    
                    for team in laps['Team'].unique():
                        team_laps = laps[laps['Team'] == team]
                        if len(team_laps) < 5:
                            continue
                        
                        try:
                            fastest = team_laps.pick_fastest()
                            tel = fastest.get_telemetry()
                            
                            if tel is None or len(tel) == 0:
                                continue
                            
                            if team not in team_data:
                                team_data[team] = []
                            
                            team_data[team].append({
                                'max_speed': tel['Speed'].max(),
                                'slow_corner_speed': tel[(tel['Speed'] >= 0) & (tel['Speed'] < 100)]['Speed'].mean(),
                                'medium_corner_speed': tel[(tel['Speed'] >= 100) & (tel['Speed'] < 200)]['Speed'].mean(),
                                'high_corner_speed': tel[(tel['Speed'] >= 200) & (tel['Speed'] < 250)]['Speed'].mean()
                            })
                        except:
                            pass
                except:
                    pass
        
        # Average and normalize
        team_avg = {}
        for team, metrics_list in team_data.items():
            df = pd.DataFrame(metrics_list)
            team_avg[team] = df.mean().to_dict()
        
        if len(team_avg) == 0:
            return None
        
        metrics = ['max_speed', 'slow_corner_speed', 'medium_corner_speed', 'high_corner_speed']
        
        avg_values = {}
        for metric in metrics:
            values = [team_avg[team][metric] for team in team_avg 
                     if not np.isnan(team_avg[team][metric])]
            avg_values[metric] = np.mean(values) if len(values) > 0 else 0
        
        directionality = {}
        for team, chars in team_avg.items():
            directionality[team] = {}
            for metric in metrics:
                value = chars[metric]
                if not np.isnan(value) and avg_values[metric] > 0:
                    directionality[team][metric] = (value - avg_values[metric]) / avg_values[metric]
                else:
                    directionality[team][metric] = 0.0
        
        print(f"  ✓ {len(directionality)} teams")
        return directionality
        
    except Exception as e:
        print(f"  Error: {e}")
        return None

def get_baseline_standings(year):
    """Get standings from previous year."""
    try:
        schedule = fastf1.get_event_schedule(year)
        last_race = schedule[schedule['EventFormat'] != 'testing'].iloc[-1]
        
        session = fastf1.get_session(year, last_race['EventName'], 'R')
        session.load(laps=False)
        
        results = session.results
        team_points = results.groupby('TeamName')['Points'].sum()
        
        max_points = team_points.max()
        if max_points > 0:
            baseline = {team: points / max_points for team, points in team_points.items()}
        else:
            baseline = {team: 0.5 for team in team_points.index}
        
        return baseline
    except:
        return {}

def calculate_track_suitability(car_directionality, track_profile):
    """Calculate track modifier."""
    total_pct = (
        track_profile['straights_pct'] +
        track_profile['slow_corners_pct'] +
        track_profile['medium_corners_pct'] +
        track_profile['high_corners_pct']
    )
    
    if total_pct == 0:
        return 0.0
    
    return (
        car_directionality.get('max_speed', 0) * (track_profile['straights_pct'] / total_pct) +
        car_directionality.get('slow_corner_speed', 0) * (track_profile['slow_corners_pct'] / total_pct) +
        car_directionality.get('medium_corner_speed', 0) * (track_profile['medium_corners_pct'] / total_pct) +
        car_directionality.get('high_corner_speed', 0) * (track_profile['high_corners_pct'] / total_pct)
    )

def get_race_results(year, race_name):
    """Get race results."""
    try:
        session = fastf1.get_session(year, race_name, 'R')
        session.load(laps=False)
        
        results = session.results[['TeamName', 'Position', 'Points']].copy()
        results = results[results['Position'].notna()]
        
        team_results = results.groupby('TeamName').agg({
            'Position': 'min',
            'Points': 'sum'
        }).sort_values('Position')
        
        return team_results
    except:
        return None

def calculate_accuracy(predicted, actual_results):
    """Spearman correlation."""
    common_teams = set([t[0] for t in predicted]) & set(actual_results.index)
    
    if len(common_teams) < 3:
        return None
    
    pred_ranks = {team: i for i, (team, _) in enumerate(predicted, 1)}
    actual_ranks = {team: int(actual_results.loc[team, 'Position']) for team in common_teams}
    
    pred_list = [pred_ranks[t] for t in common_teams]
    actual_list = [actual_ranks[t] for t in common_teams]
    
    corr, _ = spearmanr(pred_list, actual_list)
    return corr

print("✓ Functions defined")

✓ Functions defined


## Step 3: Load Data

In [5]:
# Load track profiles
track_profiles_path = Path("../data/processed/track_characteristics/track_profiles_cache.json")

if track_profiles_path.exists():
    with open(track_profiles_path) as f:
        track_profiles = json.load(f)
    print(f"✓ Loaded {len(track_profiles)} track profiles")
else:
    print("ERROR: Track profiles not found!")
    track_profiles = {}

# Get 2021 baseline
print("\nLoading 2021 baseline...")
baseline_2021 = get_baseline_standings(2021)
print(f"  ✓ {len(baseline_2021)} teams")

# Get 2022 testing
testing_2022 = extract_testing_directionality(2022)

✓ Loaded 23 track profiles

Loading 2021 baseline...
  ✓ 10 teams
Extracting 2022 testing...
  ✓ 10 teams


## Step 4: Test All Schedules

In [6]:
schedule_2022 = fastf1.get_event_schedule(2022)
races_2022 = schedule_2022[schedule_2022['EventFormat'] == 'conventional'].head(10)

schedule_results = {name: [] for name in SCHEDULES.keys()}
current_season_scores = {}

print("="*60)
print("TESTING SCHEDULES")
print("="*60)

for race_idx, (idx, race) in enumerate(races_2022.iterrows(), 1):
    race_name = race['EventName']
    
    print(f"\nRace {race_idx}: {race_name}")
    
    actual = get_race_results(2022, race_name)
    if actual is None or race_name not in track_profiles:
        print("  No data")
        continue
    
    track = track_profiles[race_name]
    
    # Update current season scores
    max_pos = actual['Position'].max()
    for team in actual.index:
        score = 1.0 - (actual.loc[team, 'Position'] - 1) / max_pos
        if team not in current_season_scores:
            current_season_scores[team] = []
        current_season_scores[team].append(score)
    
    current_season_avg = {team: np.mean(scores) for team, scores in current_season_scores.items()}
    
    # Test each schedule
    for schedule_name, weights in SCHEDULES.items():
        if race_idx not in weights:
            continue
        
        w_baseline, w_testing, w_current = weights[race_idx]
        
        predictions = {}
        
        for team in actual.index:
            baseline_score = baseline_2021.get(team, 0.5)
            
            testing_modifier = 0.0
            if team in testing_2022:
                testing_modifier = calculate_track_suitability(testing_2022[team], track)
            
            current_score = current_season_avg.get(team, baseline_score)
            
            final_score = (
                w_baseline * baseline_score +
                w_testing * testing_modifier +
                w_current * current_score
            )
            
            predictions[team] = final_score
        
        predicted = sorted(predictions.items(), key=lambda x: x[1], reverse=True)
        corr = calculate_accuracy(predicted, actual)
        
        if corr is not None:
            schedule_results[schedule_name].append({
                'race': race_idx,
                'race_name': race_name,
                'correlation': corr,
                'weights': f"{w_baseline:.0%}/{w_testing:.0%}/{w_current:.0%}"
            })
    
    # Best for this race
    best_schedule = None
    best_corr = -1
    for sched_name, results in schedule_results.items():
        if len(results) > 0 and results[-1]['race'] == race_idx:
            if results[-1]['correlation'] > best_corr:
                best_corr = results[-1]['correlation']
                best_schedule = sched_name
    
    if best_schedule:
        print(f"  Best: {best_schedule:18s} ({best_corr:.3f})")

print("\n✓ Complete")

TESTING SCHEDULES

Race 1: Bahrain Grand Prix
  Best: insane             (0.818)

Race 2: Saudi Arabian Grand Prix
  Best: ultra_aggressive   (0.855)

Race 3: Australian Grand Prix
  Best: extreme            (0.879)

Race 4: Miami Grand Prix
  Best: aggressive         (0.770)

Race 5: Spanish Grand Prix
  Best: insane             (0.927)

Race 6: Monaco Grand Prix
  Best: ultra_aggressive   (0.927)

Race 7: Azerbaijan Grand Prix
  Best: baseline_only      (0.636)

Race 8: Canadian Grand Prix
  Best: ultra_aggressive   (0.927)

Race 9: British Grand Prix
  Best: insane             (0.758)

Race 10: French Grand Prix
  No data

✓ Complete


## Step 5: Results

In [7]:
print("\n" + "="*60)
print("RESULTS")
print("="*60)

avg_correlations = {}

for schedule_name, results in schedule_results.items():
    if len(results) > 0:
        df = pd.DataFrame(results)
        avg_corr = df['correlation'].mean()
        avg_correlations[schedule_name] = avg_corr
        print(f"\n{schedule_name:18s} Avg: {avg_corr:.3f}")

if avg_correlations:
    best = max(avg_correlations.items(), key=lambda x: x[1])
    print("\n" + "="*60)
    print(f"🏆 WINNER: {best[0]} ({best[1]:.3f})")
    print("="*60)


RESULTS

conservative       Avg: 0.708

moderate           Avg: 0.714

aggressive         Avg: 0.740

very_aggressive    Avg: 0.764

ultra_aggressive   Avg: 0.790

extreme            Avg: 0.809

insane             Avg: 0.807

testing_only       Avg: 0.585

baseline_only      Avg: 0.666

🏆 WINNER: extreme (0.809)


## Step 6: Visualizations

In [8]:
# Line chart by race
fig = go.Figure()

for schedule_name, results in schedule_results.items():
    if len(results) > 0:
        df = pd.DataFrame(results)
        
        fig.add_trace(go.Scatter(
            x=df['race'],
            y=df['correlation'],
            mode='lines+markers',
            name=schedule_name,
            line=dict(width=2),
            marker=dict(size=8)
        ))

fig.add_hline(y=0.7, line_dash="dash", line_color="green", annotation_text="Strong (0.7)")
fig.add_hline(y=0.4, line_dash="dash", line_color="orange", annotation_text="Moderate (0.4)")

fig.update_layout(
    title='Prediction Accuracy by Race (2022)',
    xaxis_title='Race Number',
    yaxis_title='Spearman Correlation',
    height=600,
    hovermode='x unified'
)
fig.show()

# Bar chart of averages
if avg_correlations:
    fig2 = go.Figure(data=[
        go.Bar(
            x=list(avg_correlations.keys()),
            y=list(avg_correlations.values()),
            marker_color=['green' if v == max(avg_correlations.values()) else 'lightblue' 
                         for v in avg_correlations.values()],
            text=[f"{v:.3f}" for v in avg_correlations.values()],
            textposition='outside'
        )
    ])
    
    fig2.add_hline(y=0.7, line_dash="dash", line_color="green")
    fig2.add_hline(y=0.4, line_dash="dash", line_color="orange")
    
    fig2.update_layout(
        title='Average Correlation by Schedule (Races 1-10, 2022)',
        xaxis_title='Weight Schedule',
        yaxis_title='Average Spearman Correlation',
        height=500,
        xaxis_tickangle=-45
    )
    fig2.show()

## Conclusion

**What we learned:**

The winning schedule shows optimal weights for regulation change years.

**For 2026:**
- Use winning schedule weights
- If ultra-aggressive schedules win → trust current season VERY quickly
- If baseline helps early → team resources/infrastructure matter
- If testing helps → car DNA is predictive even during reg changes